In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("insurance.csv")
df

## EDA

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
numeric_columns = ['age', 'bmi', 'children','charges']
for col in numeric_columns:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col],kde = True,bins = 20)
    

In [ ]:
sns.countplot(x = df['children'])


In [ ]:
sns.countplot(x = df['sex'])

In [ ]:
sns.countplot(x = df['smoker'])

In [ ]:
for col in numeric_columns:
    plt.figure(figsize=(6,4))
    sns.boxplot(x = df[col])

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(numeric_only=True), annot=True)

## data cleaning and preprocessing

In [ ]:
df_cleaned = df.copy()

In [ ]:
df_cleaned.head()

In [ ]:
df.shape

In [ ]:
df_cleaned.drop_duplicates(inplace=True)

In [ ]:
df_cleaned.shape

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
df_cleaned.dtypes

In [ ]:
df_cleaned['sex'].value_counts()

In [ ]:
df_cleaned['sex'] = df_cleaned['sex'].map({'male': 0 ,'female': 1})

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned['smoker'].value_counts()

In [ ]:
df_cleaned['smoker'] = df_cleaned['smoker'].map({'no': 0 ,'yes': 1})


In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.rename(columns={
    "sex":"is_female",
    "smoker":"is_smoker"
},inplace=True)

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned['region'].value_counts()

In [ ]:
df_cleaned = pd.get_dummies(df_cleaned, columns=['region'], prefix='region', drop_first=True, dtype=int)

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned

# Feature Engineering and Extraction

In [ ]:
sns.histplot(df['bmi'])

In [ ]:
df_cleaned['bmi_category'] = pd.cut(
    df_cleaned['bmi'],
     bins=[0, 18.5, 24.9, 29.9, float('inf')],
     labels=['Underweight', 'Normal', 'Overweight', 'Obese']

)

In [ ]:
df_cleaned


In [ ]:
df_cleaned = pd.get_dummies(df_cleaned, columns=['bmi_category'], prefix='bmi_category', drop_first=True, dtype=int)


In [ ]:
df_cleaned
df2 = df_cleaned
df2

In [ ]:
df.head()

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.columns

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
cols = ['age','bmi','children']
scaler = StandardScaler()
df_cleaned[cols] = scaler.fit_transform(df_cleaned[cols])

In [ ]:
df_cleaned.head()

In [ ]:
from scipy.stats import pearsonr,chi2_contingency

# ----------------------------------
# Pearson Correlation Calculation
# ----------------------------------

# List of features to check against target
selected_features = [
    'age', 'bmi', 'children', 'is_female', 'is_smoker',
    'region_northwest', 'region_southeast', 'region_southwest',
    'bmi_category_Normal', 'bmi_category_Overweight', 'bmi_category_Obese'
]

correlations = {
    feature: pearsonr(df_cleaned[feature], df_cleaned['charges'])[0]
    for feature in selected_features
}
correlation_df = pd.DataFrame(list(correlations.items()), columns=['Feature', 'Pearson Correlation'])
correlation_df.sort_values(by='Pearson Correlation', ascending=False)

In [ ]:
cat_features = [
    'is_female', 'is_smoker',
    'region_northwest', 'region_southeast', 'region_southwest',
    'bmi_category_Normal', 'bmi_category_Overweight', 'bmi_category_Obese'
]

In [ ]:
alpha = 0.05

df_cleaned['charges_bin'] = pd.qcut(df_cleaned['charges'], q=4, labels=False)
chi2_results = {}

for col in cat_features:
    contingency = pd.crosstab(df_cleaned[col], df_cleaned['charges_bin'])
    chi2_stat, p_val, _, _ = chi2_contingency(contingency)
    decision = 'Reject Null (Keep Feature)' if p_val < alpha else 'Accept Null (Drop Feature)'
    chi2_results[col] = {
        'chi2_statistic': chi2_stat,
        'p_value': p_val,
        'Decision': decision
    }

chi2_df = pd.DataFrame(chi2_results).T
chi2_df = chi2_df.sort_values(by='p_value')
chi2_df

In [ ]:
final_df = df_cleaned[['age', 'is_female', 'bmi', 'children', 'is_smoker', 'charges','region_southeast','bmi_category_Obese']]
final_df

In [ ]:
final_df

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df2.drop('charges',axis=1)
y = df2['charges']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
np.set_printoptions(suppress=True)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2 =  r2_score(y_test, y_pred)
r2

In [ ]:
n = X_test.shape[0]
p = X_test.shape[1]

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [ ]:
adjusted_r2